# ML Enabled Mobile CGE


## 1. Building Damage Analysis

The following code is preparing the IN-CORE analysis by checking versions and connecting to IN-CORE web service.


In [ ]:
from pyincore import IncoreClient, Dataset, FragilityService, MappingSet, DataService
from pyincore.analyses.buildingdamage import BuildingDamage
from pyincore.analyses.cumulativebuildingdamage import CumulativeBuildingDamage
from pyincore.analyses.montecarlofailureprobability import MonteCarloFailureProbability

import os
import pandas as pd
import numpy as np
import geopandas as gpd  # For reading in shapefiles
import matplotlib.pyplot as plt
from IPython.display import display

import sys  # For displaying package versions
import os  # For managing directories and file paths if drive is mounted

from pyincore_viz.geoutil import GeoUtil as viz
from pyincore_viz.plotutil import PlotUtil as plot

client = IncoreClient()
client.clear_cache()

In [ ]:
# create data_service object for loading files
data_service = DataService(client)

In [ ]:
# Check package versions - good practice for replication
print("Python Version ", sys.version)
print("pandas version: ", pd.__version__)
print("numpy version: ", np.__version__)

In [ ]:
# Mobile Building Inventory from IN-CORE
Building_Inventory_id = "6827560d1d2b050c74df1526"

# visualize the building inventory
Mobile_Building_Inventory = Dataset.from_data_service(
    Building_Inventory_id, DataService(client)
)
viz.plot_map(
    Mobile_Building_Inventory, column="archetype", category=True, basemap=True
)

In [ ]:
### This CELL will be removed in the future
building_to_sectors_id = "68265d506960c84294c85f11"
building_to_sectors_df = Dataset.from_data_service(
    building_to_sectors_id, DataService(client)
).get_dataframe_from_csv()


mobile_df = Mobile_Building_Inventory.get_dataframe_from_shapefile()
# check the columns in the building inventory
print("Columns in the building inventory: ", mobile_df.columns)
# check the columns in the building to sectors
print("Columns in the building to sectors: ", building_to_sectors_df.columns)
# join based on guid
matching = mobile_df.merge(building_to_sectors_df, on="guid", how="left")
# matching = lumberton_df.join(
#     building_to_sectors_df, on="guid", how="left"
# )
print("number of buildings in the inventory: ", len(mobile_df))
print("Number of buildings in the inventory: ", len(matching))

In [ ]:
# load building inventory as Geodataframe
filename = Mobile_Building_Inventory.get_file_path("shp")
print(
    "The IN-CORE Dataservice has saved the Building Inventory on your local machine: "
    + filename
)
bldg_inv_gdf = gpd.read_file(filename)
bldg_inv_gdf.head()

In [ ]:
# Flood building archetypes mapping
#mapping_id = "63fe4ccef571b22475b6bc4f"
# Galveston Hurricane flood fragility mappings for buildings
mapping_id ="62fefd688a30d30dac57bbd7"
fragility_service = FragilityService(client)
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))

In [ ]:
# Reading the Hazard type and the Hazard ID
# hazard_type = "flood"
# hazard_id = "62fe853ff362915427002649"

# Reading the Hazard type and the Hazard ID for the six hurricane and SLR scenarios
hazard_type = "hurricane" 

hurricane_hazard_dict = {}
hurricane_hazard_dict[0] = {'id': "6420c6e6d9ae37665ff0d5f2", 'name':'Natural Katrina'} 
hurricane_hazard_dict[1] = {'id': "64222491d9ae37665ff0d5f3", 'name':'Natural Katrina + INT SLR'}
hurricane_hazard_dict[2] = {'id': "6422253cd9ae37665ff0d5f4", 'name':'Natural Katrina + EXT SLR'}
hurricane_hazard_dict[3] = {'id': "642d387bfde0f316c3493530", 'name':'Shifted Katrina'}
hurricane_hazard_dict[4] = {'id': "642d38f1fde0f316c3493531", 'name':'Shifted Katrina + INT SLR'}
hurricane_hazard_dict[5] = {'id': "642d3c7763601e1e19241468", 'name':'Shifted Katrina + EXT SLR'}

### 1.2 Building Damage Analysis


In [ ]:
bldg_dmg = BuildingDamage(client)

bldg_dmg.load_remote_input_dataset("buildings", Building_Inventory_id)
bldg_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)

In [ ]:
# # Run the building damage analysis
# result_name = "Mobile_bldg_flood_dmg_result"

# bldg_dmg.set_parameter("fragility_key", "Mobile Storm Surge flood Building Fragility ID Code")
# bldg_dmg.set_parameter("result_name", result_name)
# bldg_dmg.set_parameter("hazard_type", hazard_type)
# bldg_dmg.set_parameter("hazard_id", hazard_id)
# bldg_dmg.set_parameter("num_cpu", 4)
# bldg_dmg.run_analysis()

# Run the building damage analysis for the six hurricane and SLR scenarios

output_result=[]
output=[]
for i in range(0,len(hurricane_hazard_dict)): 
    result = []
    result2 = []
    result_name = "Mobile_bldg_natural_hurricane_dmg_result{0}".format(i)

    bldg_dmg.set_parameter("fragility_key", "Non-Retrofit Fragility ID Code")
    bldg_dmg.set_parameter("result_name", result_name)
    bldg_dmg.set_parameter("hazard_type", hazard_type)
    bldg_dmg.set_parameter("hazard_id", hurricane_hazard_dict[i]['id'])
    bldg_dmg.set_parameter("num_cpu", 8)
    bldg_dmg.run_analysis()
    # Retrieve result dataset
    result = bldg_dmg.get_output_dataset('ds_result')
    # Convert dataset to Pandas DataFrame
    result2 = result.get_dataframe_from_csv(low_memory=False)
    output.append(result2)
    output_result.append(result)
building_dmg_result_MC = output_result
building_dmg_result = output

The following code explores the buildings damage analysis results


In [ ]:
# Select only the buildings exposed to the hazard sceanrios
output3=[]
for j in range(0,len(building_dmg_result)):
    output2 = building_dmg_result[j][building_dmg_result[j]['haz_expose']=='yes']
    output3.append(output2)
bdmg_df_expose = output3
    
    
# Add 'DS_max' attribute to bdmg_df_expose that provide the max damage state for each Mobile residential building
for j in range(0,len(bdmg_df_expose)):
    bdmg_df_expose[j]['DS_max'] = bdmg_df_expose[j].loc[:,['DS_0', 'DS_1', 'DS_2', 'DS_3']].idxmax(axis = 1)

In [ ]:
output5=[]
output7=[]
for k in range(0,len(bdmg_df_expose)):
    output4 = bdmg_df_expose[k]['DS_max'].value_counts(normalize=True).mul(100).index.tolist()
    output6 = bdmg_df_expose[k]['DS_max'].value_counts(normalize=True).mul(100).tolist()
    output5.append(output4)
    output7.append(output6)
    indexes = output5
    values  = output7
    

# Set plot parameters
fig, ax = plt.subplots(figsize=(20, 10), dpi=300)
width = 0.15 # width of bar
x = np.arange(len(indexes[0]))
x_labels = ['DS_0', 'DS_1', 'DS_2', 'DS_3']


containers = ax.bar(x , values[0], width, color='#000080', label='Natural Katrina')
containers = ax.bar(x  + width, values[1], width, color='#0F52BA', label='Natural Katrina + INT SLR')
containers = ax.bar(x  + (2 * width), values[2], width, color='#6593F5', label='Natural Katrina + EXT SLR')
containers = ax.bar(x  + (3 * width), values[3], width, color='#73C2FB', label='Shifted Katrina')
containers = ax.bar(x  + (4 * width), values[4], width, color='#81C6EB', label='Shifted Katrina + INT SLR')
containers = ax.bar(x  + (5 * width), values[5], width, color='#D4F1F7', label='Shifted Katrina + EXT SLR')



for bars in ax.containers:
    ax.bar_label(bars, labels = [f'{x.get_height()/100:.1%}' for x in bars], padding=5, fontsize = 15)
    
    
ax.set_ylabel('Percentage of the buildings (%)', labelpad=15, fontsize = 20)
ax.set_ylim(0,110)
ax.set_xticks(x + width + width/2)
ax.set_xticklabels(x_labels, fontsize = 20)
ax.set_xlabel('Damage State', labelpad=15, fontsize = 20)
ax.set_title('Distribution of most probable damage state for residential buildings',fontsize = 25)
ax.legend(fontsize = 20)
# ax('ytick', labelsize=10)    # fontsize of the tick labels
plt.tick_params(labelsize=20)
plt.grid(True, 'major', 'y', ls='--', lw=.5, c='k', alpha=.3)

fig.tight_layout()
plt.show()

In [ ]:
# To visualize the output of Natural Katrina

bldg_results = pd.merge(bldg_inv_gdf, building_dmg_result[2], how = 'right', left_on = ['guid'], right_on=['guid'])

In [ ]:
# To show the probability of exceedance of DS_1 > 0

bldg_results_F1 = bldg_results[(bldg_results['archetype'] > 0) & (bldg_results['DS_1'] > 0) ]
bldg_results_F1.explore(column='DS_1',cmap='Reds', 
                        popup=['guid','g_elev','ffe_elev','LS_0','LS_1','LS_2','DS_1','DS_2'
                                ],
                        tooltip=['guid','g_elev','ffe_elev','LS_0','LS_1','LS_2','DS_1','DS_2'
                                ],
                        tiles='CartoDB positron',
                        style_kwds=dict(color="Red",weight=5, opacity=0.4))

# Retrieve result dataset
building_dmg_result = bldg_dmg.get_output_dataset("ds_result")

In [ ]:
# Convert dataset to Pandas DataFrame
bdmg_df = building_dmg_result.get_dataframe_from_csv(low_memory=False)

# Display top 5 rows of output data
bdmg_df.head()

## 2. Montecarlo Limit State Probability


In [ ]:
from pyincore.analyses.montecarlofailureprobability import MonteCarloFailureProbability

In [ ]:
output_mc=[]
num_samples = 500

for j in range(0,len(building_dmg_result_MC)):

    mc_bldg = MonteCarloLimitStateProbability(client)

    mc_bldg.set_input_dataset("damage", building_dmg_result_MC[j])  #  Load the Mobile building damage results dataset 
                                                             # generated from the previous model as an input
    mc_bldg.set_parameter("num_cpu", 8)
    mc_bldg.set_parameter("num_samples", num_samples)
    mc_bldg.set_parameter("damage_interval_keys", ["DS_0", "DS_1", "DS_2", "DS_3"])
    mc_bldg.set_parameter("failure_state_keys", ["DS_1", "DS_2", "DS_3"])

    mc_bldg.set_parameter("result_name", "storm_surge_mc_failure_probability_buildings") # name of csv file with results
    mc_bldg.run_analysis()  # Run the Monte Carlo Simulation module to obtain the building failure probabilities. 
    building_failure_probability = mc_bldg.get_output_dataset('failure_probability')  # get buildings failure probabilities
    result_mc = building_failure_probability.get_dataframe_from_csv()
    output_mc.append(result_mc)
df_bldg_fail = output_mc

In [ ]:
df_bldg_fail[0][df_bldg_fail[0]['failure_probability']>0]

In [ ]:
mc_bldg.run_analysis()

# Obtain buildings failure probabilities
building_failure_probability = mc_bldg.get_output_dataset("failure_probability")

df_bldg_fail = building_failure_probability.get_dataframe_from_csv()
df_bldg_fail.head()

In [ ]:
# Obtain buildings sample damage states
sample_damage_states = mc_bldg.get_output_dataset("sample_damage_states")
df_bldg_ds = sample_damage_states.get_dataframe_from_csv()
df_bldg_ds.head()

In [ ]:
building_damage_mcs_samples = mc_bldg.get_output_dataset(
    "sample_failure_state"
)  # get buildings failure states

bdmcs = building_damage_mcs_samples.get_dataframe_from_csv()
bdmcs.head()

# 3. Capital Stock Shock and CGE Analysis


In [ ]:
from pyincore.analyses.capitalshocks import CapitalShocks

In [ ]:
# Joplin building to sector mapping table
building_to_sectors_id = "682dfb711d2b050c74e02e55"
# Create Capital shocks analysis
capital_shocks = CapitalShocks(client)

# Load remote datasets
capital_shocks.load_remote_input_dataset("buildings_to_sectors", building_to_sectors_id)
# Set datasets
# Joplin building inventory
capital_shocks.load_remote_input_dataset("buildings", Building_Inventory_id)
# Joplin building failure probability
capital_shocks.set_input_dataset("failure_probability", building_failure_probability)

capital_shocks.set_parameter(
    "result_name", "sector_shocks"
)  # name of csv file with results

In [ ]:
# Run capital shocks analysis
capital_shocks.run_analysis()

sector_shocks_result = capital_shocks.get_output_dataset("sector_shocks")
sector_shocks_result.get_dataframe_from_csv()

### 4. ML Enabled CGE - Lumberton


In [ ]:
from pyincore.analyses.mlenabledcgelumberton import MlEnabledCgeLumberton

In [ ]:
mlcgelumberton = MlEnabledCgeLumberton(client)
mlcgelumberton.set_input_dataset("sector_shocks", sector_shocks_result)
mlcgelumberton.set_parameter("result_name", "test_lumberton_mlcge_result")

In [ ]:
mlcgelumberton.run_analysis()